###**Intrinsic Functions**
- Specialized functions provided by the CUDA programming model. They are callable only from the device. They do not need to include any additional headers in your program.
- These functions often offer an alternative to standard functions that are faster but may have less numerical accuracy, they are majorly used in mathematical functions.


### **Thread Synchronization**

Threads **within a block** can cooperate by sharing data through some shared memory and by synchronizing their execution to coordinate memory accesses. More precisely, one can specify synchronization points in the kernel by calling the __syncthreads() **intrinsic** function; __syncthreads() acts as a barrier at which all threads in the block must wait before any is allowed to proceed

In [1]:
%%writefile reverse1.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

__global__ void staticReverse(int *d, int n)
{
  __shared__ int s[64];
  int t = threadIdx.x;
  int tr = n-t-1;
  s[t] = d[t];
  __syncthreads();
  d[t] = s[tr];
}


int main(void)
{
  const int n = 64;
  int a[n], r[n], d[n];

  for (int i = 0; i < n; i++) {
    a[i] = i;
    r[i] = n-i-1;
    d[i] = 0;
  }

  int *d_d;
  cudaMalloc(&d_d, n * sizeof(int)); 

  // run version with static shared memory
  cudaMemcpy(d_d, a, n*sizeof(int), cudaMemcpyHostToDevice);
  staticReverse<<<1,n>>>(d_d, n);
  cudaMemcpy(d, d_d, n*sizeof(int), cudaMemcpyDeviceToHost);
  for (int i = 0; i < n; i++) 
    if (d[i] != r[i]) printf("Error: d[%d]!=r[%d] (%d, %d)n", i, i, d[i], r[i]);
}

Writing reverse1.cu


In [ ]:
%%writefile reverse2.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

__global__ void dynamicReverse(int *d, int n)
{
  extern __shared__ int s[];
  int t = threadIdx.x;
  int tr = n-t-1;
  s[t] = d[t];
  __syncthreads();
  d[t] = s[tr];
}

int main(void)
{
  const int n = 64;
  int a[n], r[n], d[n];

  for (int i = 0; i < n; i++) {
    a[i] = i;
    r[i] = n-i-1;
    d[i] = 0;
  }

  int *d_d;
  cudaMalloc(&d_d, n * sizeof(int)); 

  // run dynamic shared memory version
  cudaMemcpy(d_d, a, n*sizeof(int), cudaMemcpyHostToDevice);
  dynamicReverse<<<1,n,n*sizeof(int)>>>(d_d, n);
  cudaMemcpy(d, d_d, n * sizeof(int), cudaMemcpyDeviceToHost);
  for (int i = 0; i < n; i++) 
    if (d[i] != r[i]) printf("Error: d[%d]!=r[%d] (%d, %d)n", i, i, d[i], r[i]);
}

#### **Dynamic Shared Memory**

Can be used when the amount of shared memory is not known at compile time. In this case the shared memory allocation size per thread block must be specified (in bytes) using an optional third execution configuration parameter, as follows:

*dynamicReverse<<<1, n, n* * *sizeof(int)>>>(d_d, n)*

What if you need multiple dynamically sized arrays in a single kernel? You must declare a single extern unsized array as before, and use pointers into it to divide it into multiple arrays, as follows:

In [ ]:
extern __shared__ int s[];
int *integerData = s;                        // nI ints
float *floatData = (float*)&integerData[nI]; // nF floats
char *charData = (char*)&floatData[nF];      // nC chars

// In the kernel launch, specify the total shared memory needed:
myKernel<<<gridSize, blockSize, nI*sizeof(int)+nF*sizeof(float)+nC*sizeof(char)>>>(...);

The __syncthreads() statement in the for-loop ensures that all partial sums for the previous iteration have been generated and before any one of the threads is allowed to begin the current iteration

#### **Thread Divergence**

In [ ]:
# Consider this example

# Does this code work properly? why?
if{
     ...
     __syncthreads();
}else{
     ...
     __syncthreads();
}

If a thread in a block executes the then-path and another executes the else-path, they would be waiting at different barrier synchronization points and end up waiting for each other forever. so if __syncthreads() exists in the kernel, it must be executed by all threads. In this sense, the above code can be fixed as follows:

In [ ]:
if{
     ...
}
else{
     ...
}
__syncthreads();

### **Let's Practise Together**

modify the last requirement so that the last 1d vector summation is done using the following ways and show their profiling:
- Use only 1 block for your kernel and let the CPU handle the final sum.
- Use only 1 block for your kernal and let one thread to handle the final sum.
- Use multiple blocks for your kernal and let the CPU handle the final sum.

IF YOU HAVE ANY INQUIRIES PLEASE DO NOT HESITATE TO CONTACT ME
